# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import matplotlib

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,-1.15,86,75,5.14,CA,1684981953
1,1,cuito,-12.3833,16.9333,12.05,46,13,1.03,AO,1684981953
2,2,qulsary,46.9531,54.0198,23.71,23,59,8.28,KZ,1684981953
3,3,maumere,-8.6199,122.2111,28.77,67,43,2.30,ID,1684981953
4,4,san antonio de pale,-1.4014,5.6325,26.35,88,98,6.16,GQ,1684981944


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    c='Humidity',
    title='City Map',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['City'],
    size='Humidity',
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down the cities to fit weather conditions"
#A max temperature lower than 27 degrees but higher than 21
temp_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
wind_df = temp_df[temp_df["Wind Speed"] < 4.5]

# Zero cloudiness:
cloud_df = wind_df[wind_df["Cloudiness"] == 0]


# Cloud_df any rows will null values:
cloud_df = cloud_df.dropna()
cloud_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cloud_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
81,makakilo city,US,21.3469,-158.0858,81,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
211,laguna,US,38.4210,-121.4238,79,
240,tikaitnagar,IN,26.9500,81.5833,59,
265,san quintin,MX,30.4833,-115.9500,74,
340,santa rosalia,MX,27.3167,-112.2833,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
filters = {
    "radius": radius
}
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Update the "bias" parameter in the params dictionary with the current city's latitude and longitude
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name,Hover Info
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort,"kapaa, No hotel found, US"
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel,"hilo, No hotel found, US"
63,banda,IN,25.4833,80.3333,52,#acnindiafy21,"banda, No hotel found, IN"
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei,"makakilo city, No hotel found, US"
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel,"kahului, No hotel found, US"
197,gat,IL,31.6100,34.7642,100,No hotel found,"gat, No hotel found, IL"
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites,"laguna, No hotel found, US"
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found,"tikaitnagar, No hotel found, IN"
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel,"san quintin, No hotel found, MX"
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar,"santa rosalia, No hotel found, MX"


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    c='Humidity',
    title='City Map',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)